In [ ]:
import pandas as pd

In [ ]:
# load all data as CSV
# I think if you want to be fancy you can actually do a query within the UI or use their API to request a subset of rows
ridership = pd.read_csv('https://data.cityofchicago.org/api/views/jyb9-n7fm/rows.csv?accessType=DOWNLOAD')

In [ ]:
# convert to actual date
ridership['date'] = pd.to_datetime(ridership.date, format="%m/%d/%Y")

In [ ]:
ridership.rename({'route': 'route_id'}, axis = 1, inplace = True)

In [ ]:
# take July 2022 subset
july = ridership[(ridership['date'].dt.month == 7) & (ridership['date'].dt.year == 2022)].copy()

In [ ]:
# map their encoded values
july['day_type'] = july.daytype.map({'W': 'weekday', 'A': 'sat', 'U': 'sun'})

In [ ]:
# handle 4th of July
july.loc[july.date == '2022-07-04', 'day_type'] = 'hol'

In [ ]:
# raw daily data
july[['route_id', 'date', 'day_type', 'rides']].to_json('daily_july2022_cta_ridership_data.json',orient = 'records')

In [ ]:
# group to get day type averages
july_daytype_summary = july.groupby(by = ['route_id', 'day_type']).agg({'rides': ['mean', 'sum']}).reset_index()

In [ ]:
# reset from multiindex
july_daytype_summary.columns = ['route_id', 'day_type', 'avg_riders', 'total_riders']

In [ ]:
# summary by day type
july_daytype_summary.to_json('july2022_cta_ridership_data_day_type_summary.json',orient = 'records')